Sentiment Analysis

In [2]:
import zipfile, os

def load_labeled_reviews(zip_path, extract_to="dataset"):
    reviews = []
    labels = []

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    for root, _, files in os.walk(extract_to):
        for file in files:
            if file.endswith(".txt"):
                label = 1 if "pos" in root.lower() else 0
                file_path = os.path.join(root, file)
                with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                    text = f.read().strip()
                    if text:  # skip empty
                        reviews.append(text)
                        labels.append(label)

    return reviews, labels


In [4]:
reviews, labels = load_labeled_reviews("comments2k-1.zip")
print(f"Loaded {len(reviews)} reviews — {sum(labels)} positive, {len(labels) - sum(labels)} negative")


Loaded 4000 reviews — 2000 positive, 2000 negative


In [5]:
import random

# Combine and shuffle to randomize
combined = list(zip(reviews, labels))
random.seed(42)
random.shuffle(combined)

# Few-shot examples
few_shot = combined[:20]  # 10 pos + 10 neg
few_shot_texts, few_shot_labels = zip(*few_shot)

# Remaining for test
test_data = combined[20:]
test_texts, test_labels = zip(*test_data)
print(f"Test:     {len(test_texts)} examples")


Test:     3980 examples


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load FLAN-T5 base model
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.eval().to("cuda")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [7]:
import torch
def classify_sentiment(prompt, max_length=32):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip().lower()



In [8]:
def zero_shot_prompt(text):
    return f"Review: {text}\nSentiment:"

def few_shot_prompt(text, k=10):
    prompt = ""
    count_pos = count_neg = 0
    for review, label in zip(few_shot_texts, few_shot_labels):
        if label == 1 and count_pos < k // 2:
            prompt += f"Review: {review}\nSentiment: positive\n"
            count_pos += 1
        elif label == 0 and count_neg < k // 2:
            prompt += f"Review: {review}\nSentiment: negative\n"
            count_neg += 1
    prompt += f"Review: {text}\nSentiment:"
    return prompt


In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_preds(preds, true_labels, name="Model"):
    acc = accuracy_score(true_labels, preds)
    prec = precision_score(true_labels, preds)
    rec = recall_score(true_labels, preds)
    f1 = f1_score(true_labels, preds)
    cm = confusion_matrix(true_labels, preds)

    print(f"📊 {name} Results:")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print()


In [23]:
zero_preds = []
for i, text in enumerate(test_texts):
    if i % 100 == 0:
        print(f"Processing {i}/{len(test_texts)}")
    pred = classify_sentiment(zero_shot_prompt(text))
    zero_preds.append(1 if "pos" in pred else 0)


Processing 0/3980
Processing 100/3980
Processing 200/3980
Processing 300/3980
Processing 400/3980
Processing 500/3980
Processing 600/3980
Processing 700/3980
Processing 800/3980
Processing 900/3980
Processing 1000/3980
Processing 1100/3980
Processing 1200/3980
Processing 1300/3980
Processing 1400/3980
Processing 1500/3980
Processing 1600/3980
Processing 1700/3980
Processing 1800/3980
Processing 1900/3980
Processing 2000/3980
Processing 2100/3980
Processing 2200/3980
Processing 2300/3980
Processing 2400/3980
Processing 2500/3980
Processing 2600/3980
Processing 2700/3980
Processing 2800/3980
Processing 2900/3980
Processing 3000/3980
Processing 3100/3980
Processing 3200/3980
Processing 3300/3980
Processing 3400/3980
Processing 3500/3980
Processing 3600/3980
Processing 3700/3980
Processing 3800/3980
Processing 3900/3980


In [24]:
evaluate_preds(zero_preds, test_labels, "Zero-shot FLAN-T5 (3980 reviews)")


📊 Zero-shot FLAN-T5 (3980 reviews) Results:
Accuracy:  0.5013
Precision: 0.5000
Recall:    0.0050
F1 Score:  0.0100
Confusion Matrix:
[[1985   10]
 [1975   10]]



In [25]:
ten_shot_preds = []
for i, text in enumerate(test_texts):
    if i % 100 == 0:
        print(f"10-shot: Processing {i}/{len(test_texts)}")
    prompt = few_shot_prompt(text, k=10)
    pred = classify_sentiment(prompt)
    ten_shot_preds.append(1 if "pos" in pred else 0)

evaluate_preds(ten_shot_preds, test_labels, "10-shot FLAN-T5 (3980 reviews)")


10-shot: Processing 0/3980
10-shot: Processing 100/3980
10-shot: Processing 200/3980
10-shot: Processing 300/3980
10-shot: Processing 400/3980
10-shot: Processing 500/3980
10-shot: Processing 600/3980
10-shot: Processing 700/3980
10-shot: Processing 800/3980
10-shot: Processing 900/3980
10-shot: Processing 1000/3980
10-shot: Processing 1100/3980
10-shot: Processing 1200/3980
10-shot: Processing 1300/3980
10-shot: Processing 1400/3980
10-shot: Processing 1500/3980
10-shot: Processing 1600/3980
10-shot: Processing 1700/3980
10-shot: Processing 1800/3980
10-shot: Processing 1900/3980
10-shot: Processing 2000/3980
10-shot: Processing 2100/3980
10-shot: Processing 2200/3980
10-shot: Processing 2300/3980
10-shot: Processing 2400/3980
10-shot: Processing 2500/3980
10-shot: Processing 2600/3980
10-shot: Processing 2700/3980
10-shot: Processing 2800/3980
10-shot: Processing 2900/3980
10-shot: Processing 3000/3980
10-shot: Processing 3100/3980
10-shot: Processing 3200/3980
10-shot: Processing 33

In [10]:
twenty_shot_preds = []
for i, text in enumerate(test_texts):
    if i % 100 == 0:
        print(f"20-shot: Processing {i}/{len(test_texts)}")
    prompt = few_shot_prompt(text, k=20)
    pred = classify_sentiment(prompt)
    twenty_shot_preds.append(1 if "pos" in pred else 0)

evaluate_preds(twenty_shot_preds, test_labels, "20-shot FLAN-T5 (3980 reviews)")


20-shot: Processing 0/3980
20-shot: Processing 100/3980
20-shot: Processing 200/3980
20-shot: Processing 300/3980
20-shot: Processing 400/3980
20-shot: Processing 500/3980
20-shot: Processing 600/3980
20-shot: Processing 700/3980
20-shot: Processing 800/3980
20-shot: Processing 900/3980
20-shot: Processing 1000/3980
20-shot: Processing 1100/3980
20-shot: Processing 1200/3980
20-shot: Processing 1300/3980
20-shot: Processing 1400/3980
20-shot: Processing 1500/3980
20-shot: Processing 1600/3980
20-shot: Processing 1700/3980
20-shot: Processing 1800/3980
20-shot: Processing 1900/3980
20-shot: Processing 2000/3980
20-shot: Processing 2100/3980
20-shot: Processing 2200/3980
20-shot: Processing 2300/3980
20-shot: Processing 2400/3980
20-shot: Processing 2500/3980
20-shot: Processing 2600/3980
20-shot: Processing 2700/3980
20-shot: Processing 2800/3980
20-shot: Processing 2900/3980
20-shot: Processing 3000/3980
20-shot: Processing 3100/3980
20-shot: Processing 3200/3980
20-shot: Processing 33

In [11]:
print(few_shot_prompt(test_texts[0], k=10))


Review:     Mac OS X            	   2                                               ATTR             >                     >  com.apple.quarantine q/0081;64e039bc;sharingd;0B546ADD-055C-4E77-97E6-CBDC308B9D39 
Sentiment: negative
Review: I Am Curious is really two films in one - half of it is the sexual experimental side of Lena and the other half is her curiosity with political/socialism. Whatever the director's intention, the two don't really mesh together. The director should have just stuck with the romantic side of Lena and made a separate movie for the politics. There is a bizarre mixture of political/war rallies, Dr. King, serious political interviews, flopping breasts, and pubic hair. The film feels more like a fictional documentary than a movie. Other than the interesting sex scenes, you'll be bored dry watching this film. Unlike many other reviewers, I think the nude/sexual scenes are overdone for what it is. If you want to see real porn, I'm sure there are better cho